In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import sklearn
import nltk
import re 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfVectorizer #as vectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.linear_model import LogisticRegression

In [3]:
training = pd.read_csv('/content/gdrive/My Drive/NMSM/day2/train.csv', encoding="utf-8")
testing  = pd.read_csv('/content/gdrive/My Drive/NMSM/day2/test.csv', encoding="utf-8")
print (training.head(5))
print (testing.head(5))
print (testing.tail(5))

  restaurant_id  ... Sentiment
0          tr70  ...  positive
1          tr70  ...  positive
2          tr70  ...  positive
3          tr70  ...  positive
4          tr70  ...  positive

[5 rows x 6 columns]
  restaurant_id  ... Sentiment
0         ts106  ...  negative
1         ts106  ...  negative
2         ts106  ...  negative
3         ts106  ...  negative
4         ts106  ...  negative

[5 rows x 6 columns]
     restaurant_id  ... Sentiment
3792          ts56  ...  negative
3793          ts56  ...  positive
3794          ts56  ...  positive
3795          ts56  ...  negative
3796          ts56  ...  positive

[5 rows x 6 columns]


In [4]:
#Label conversion: Positive to 1,Negative to -1 
train_pos = training[(training.Sentiment == 'positive')]
train_neg = training[(training.Sentiment == 'negative')]
print (train_pos.head(3))
print (train_pos.head(3))

train_pos_list = []
for i,t in train_pos.iterrows():
    train_pos_list.append([t.text.lower(), 1])

train_neg_list = []
for i,t in train_neg.iterrows():
    train_neg_list.append([t.text.lower(), -1])

  restaurant_id  ... Sentiment
0          tr70  ...  positive
1          tr70  ...  positive
2          tr70  ...  positive

[3 rows x 6 columns]
  restaurant_id  ... Sentiment
0          tr70  ...  positive
1          tr70  ...  positive
2          tr70  ...  positive

[3 rows x 6 columns]


In [18]:
import pandas as pd
import pickle as pk
#Same for test dataset   
test_pos = testing[(testing.Sentiment == 'positive')]
test_neg = testing[(testing.Sentiment == 'negative')]

test_pos_list = []
for i,t in test_pos.iterrows():
    test_pos_list.append([t.text.lower(), 1])

test_neg_list = []
for i,t in test_neg.iterrows():
    test_neg_list.append([t.text.lower(), -1])

#build the two dataset
trainset = train_pos_list + train_neg_list
testset = test_pos_list + test_neg_list

pk.dump(trainset, open('/content/gdrive/My Drive/NMSM/day2/trainset.pk', "wb"))
pk.dump(testset , open('/content/gdrive/My Drive/NMSM/day2/testset.pk', "wb"))


In [9]:
##########################################################3
###Preprocessing 
# seperate the text with labels

X_train = [t[0] for t in trainset]
X_test = [t[0] for t in testset]

Y_train = [t[1] for t in trainset]
Y_test = [t[1] for t in testset]

#Vectorizer the sentences using Tfidf vale
#Make sure test data should be transformed using vectorizer learned from trainning data 
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

# same feature set
train_vectors.shape
test_vectors.shape

(3797, 549262)

In [10]:
#########################################################
#Apply NB model
clf_NB = MultinomialNB().fit(train_vectors, Y_train)
predNB = clf_NB.predict(test_vectors)
pred = list(predNB)
Y_test

'''
errors=[]
for a,b, index in pred,Y_test:
    if a==b:
        continue
    else:
        errors.append(test_set[index])
print (errors)    
'''

print(metrics.confusion_matrix(Y_test, pred))
print(metrics.classification_report(Y_test, pred))

[[1758   39]
 [1237  763]]
              precision    recall  f1-score   support

          -1       0.59      0.98      0.73      1797
           1       0.95      0.38      0.54      2000

    accuracy                           0.66      3797
   macro avg       0.77      0.68      0.64      3797
weighted avg       0.78      0.66      0.63      3797



In [11]:
# MaxEnt = LogisticRegression
clf_ME = LogisticRegression(random_state=0, solver='lbfgs').fit(train_vectors, Y_train)
predME = clf_ME.predict(test_vectors)
pred = list(predME)
print(metrics.confusion_matrix(Y_test, pred))
print(metrics.classification_report(Y_test, pred))

#####KNN Classifier
def train_knn(X, y, k, weight):
    """
    Create and train the k-nearest neighbor.
    """
    knn = KNeighborsClassifier(n_neighbors = k, weights = weight, metric = 'cosine', algorithm = 'brute')
    knn.fit(X, y)
    return knn


kn = train_knn(train_vectors, Y_train, 3, 'distance')# distance weights - by inverse of distance
predKN = kn.predict(test_vectors)
pred = list(predKN)
print(metrics.confusion_matrix(Y_test, pred))
print(metrics.classification_report(Y_test, pred))


[[1589  208]
 [ 350 1650]]
              precision    recall  f1-score   support

          -1       0.82      0.88      0.85      1797
           1       0.89      0.82      0.86      2000

    accuracy                           0.85      3797
   macro avg       0.85      0.85      0.85      3797
weighted avg       0.86      0.85      0.85      3797

[[1377  420]
 [ 616 1384]]
              precision    recall  f1-score   support

          -1       0.69      0.77      0.73      1797
           1       0.77      0.69      0.73      2000

    accuracy                           0.73      3797
   macro avg       0.73      0.73      0.73      3797
weighted avg       0.73      0.73      0.73      3797



In [12]:
#Apply SVM model

from sklearn import svm
from sklearn.svm import LinearSVC

model_svm = LinearSVC(C=100)
clr_svm = model_svm.fit(train_vectors, Y_train)


predicted = clr_svm.predict(test_vectors)
 
print(metrics.confusion_matrix(Y_test, predicted))
print(np.mean(predicted == Y_test) )
print(metrics.classification_report(Y_test, predicted))

[[1513  284]
 [ 277 1723]]
0.8522517777192521
              precision    recall  f1-score   support

          -1       0.85      0.84      0.84      1797
           1       0.86      0.86      0.86      2000

    accuracy                           0.85      3797
   macro avg       0.85      0.85      0.85      3797
weighted avg       0.85      0.85      0.85      3797



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [13]:
#ADD Features - Negation
import re 
def nega_tag(text):
    transformed = re.sub(r"\b(?:never|nothing|nowhere|noone|none|not|haven't|hasn't|hasnt|hadn't|hadnt|can't|cant|couldn't|couldnt|shouldn't|shouldnt|won't|wont|wouldn't|wouldnt|don't|dont|doesn't|doesnt|didn't|didnt|isnt|isn't|aren't|arent|aint|ain't|hardly|seldom)\b[\w\s]+[^\w\s]", lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0)), text, flags=re.IGNORECASE)
    return(transformed)

text = "I don't like that place , you keep calling awesome."
print (nega_tag(text))

# Create a training list which will now contain reviews with Negatively tagged words and their labels
train_set_nega = []

# Append elements to the list
for doc in trainset:
    trans = nega_tag(doc[0])
    lab = doc[1]
    train_set_nega.append([trans, lab])

print(train_set_nega[18])

# Create a testing list which will now contain reviews with Negatively tagged words and their labels
test_set_nega = []

# Append elements to the list
for doc in testset:
    trans = nega_tag(doc[0])
    lab = doc[1]
    test_set_nega.append([trans, lab])


#Redo - Preprocessing 
# seperate the text with labels


X_nega_train = [t[0] for t in train_set_nega]
X_nega_test = [t[0] for t in test_set_nega]

Y_nega_train = [t[1] for t in train_set_nega]
Y_nega_test = [t[1] for t in test_set_nega]

#Vectorizer the sentences using Tfidf vale
#Make sure test data should be transformed using vectorizer learned from trainning data 
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
train_nega_vectors = vectorizer.fit_transform(X_nega_train)
test_nega_vectors = vectorizer.transform(X_nega_test)

# bigger feature set
train_vectors.shape
test_vectors.shape

train_nega_vectors.shape
test_nega_vectors.shape

I don't NEG_like NEG_that NEG_place , you keep calling awesome.
["another restaurant kevin and i frequent when we are out and about in chandler. good prices, and the food is consistently good. i love the vegetarian pizza. in fact, their pizzas are all pretty good. kevin got the nachos last time, which he enjoyed. i am no beer critic, since i pretty much hate it, but since it is a brewery, i can only assume the beer is up to par. the service is also good, and i love how lively it always is, even at ten on a monday night. i always walk away satisfied and with the confidence that i have not NEG_just NEG_wasted NEG_my NEG_money NEG_on NEG_mediocre NEG_food NEG_and NEG_drinks. thanks for being a friend, bj's!", 1]


(3797, 610107)

In [14]:
#Re-train NB model
clf_NB = MultinomialNB().fit(train_nega_vectors, Y_nega_train)
predNB = clf_NB.predict(test_nega_vectors)
pred = list(predNB)

print(metrics.confusion_matrix(Y_nega_test, pred))
print(np.mean(pred == Y_nega_test) )
print(metrics.classification_report(Y_nega_test, pred))

[[1755   42]
 [1128  872]]
0.0
              precision    recall  f1-score   support

          -1       0.61      0.98      0.75      1797
           1       0.95      0.44      0.60      2000

    accuracy                           0.69      3797
   macro avg       0.78      0.71      0.67      3797
weighted avg       0.79      0.69      0.67      3797



In [15]:
#Re-train KNN
kn = train_knn(train_nega_vectors, Y_nega_train, 20, 'distance')# distance weights - by inverse of distance
predKN = kn.predict(test_nega_vectors)
pred = list(predKN)
print(metrics.confusion_matrix(Y_nega_test, pred))
print(metrics.classification_report(Y_nega_test, pred))


[[1474  323]
 [ 489 1511]]
              precision    recall  f1-score   support

          -1       0.75      0.82      0.78      1797
           1       0.82      0.76      0.79      2000

    accuracy                           0.79      3797
   macro avg       0.79      0.79      0.79      3797
weighted avg       0.79      0.79      0.79      3797



In [16]:
#Re-train the SVM
model_svm = LinearSVC(C=1.0)
clr_svm = model_svm.fit(train_nega_vectors, Y_nega_train)    
predicted_nega = clr_svm.predict(test_nega_vectors)
 
print(metrics.confusion_matrix(Y_nega_test, predicted_nega))
print(np.mean(predicted_nega == Y_nega_test) )
print(metrics.classification_report(Y_nega_test, predicted_nega))

[[1516  281]
 [ 247 1753]]
0.8609428496181196
              precision    recall  f1-score   support

          -1       0.86      0.84      0.85      1797
           1       0.86      0.88      0.87      2000

    accuracy                           0.86      3797
   macro avg       0.86      0.86      0.86      3797
weighted avg       0.86      0.86      0.86      3797



In [17]:
##Select K Best features

ch21 = SelectKBest(chi2, k=600)
# Transform your training and testing datasets accordingly
train_Kbest = ch21.fit_transform(train_nega_vectors, Y_nega_train)
test_Kbest = ch21.transform(test_nega_vectors)
train_Kbest.shape

model_svm = LinearSVC(C=1.0)
# Train your SVM with the k best selected features
sv = model_svm.fit(train_Kbest, Y_nega_train)
predSVM= sv.predict(test_Kbest)
pred = list(predSVM)

print(metrics.confusion_matrix(Y_nega_test, pred))
print(np.mean(predSVM == Y_nega_test) )
print(metrics.classification_report(Y_nega_test, pred))

selected_features = list(np.array(vectorizer.get_feature_names())[ch21.get_support()])
print (selected_features)

#Re-Train ME model
clf_ME = LogisticRegression(random_state=0, solver='lbfgs').fit(train_Kbest, Y_nega_train)
predME = clf_ME.predict(test_Kbest)
pred = list(predME)
print(metrics.confusion_matrix(Y_test, pred))
print(metrics.classification_report(Y_test, pred))

[[1566  231]
 [ 333 1667]]
0.8514616802739005
              precision    recall  f1-score   support

          -1       0.82      0.87      0.85      1797
           1       0.88      0.83      0.86      2000

    accuracy                           0.85      3797
   macro avg       0.85      0.85      0.85      3797
weighted avg       0.85      0.85      0.85      3797



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['10 minutes', '15', '20', '20 minutes', 'absolutely delicious', 'affordable', 'after', 'also have', 'always', 'always good', 'amazing', 'amazing and', 'amazing food', 'amazing the', 'and delicious', 'and friendly', 'and good', 'and great', 'and left', 'and said', 'another', 'are always', 'are great', 'arugula', 'as good', 'as well', 'asked', 'asked for', 'asked if', 'at all', 'at best', 'atmosphere', 'attentive', 'authentic', 'average', 'awesome', 'awesome and', 'awful', 'back again', 'bad', 'bad service', 'bagel', 'bagels', 'bar bianco', 'bar next', 'barely', 'bass', 'be back', 'because', 'belly', 'best', 'best have', 'best in', 'best pizza', 'best thai', 'best ve', 'better', 'bianco', 'biancoverde', 'bill', 'bland', 'bland and', 'brazilian', 'bun', 'burger', 'burger was', 'burgers', 'burnt', 'but worth', 'cab', 'came', 'can neg_wait', 'card', 'catfish', 'champagne', 'charge', 'charged', 'chris', 'closed', 'coconut', 'coconut ice', 'cold', 'cold and', 'crispy', 'crispy duck', 'crispy